In [50]:
# %pip install ipympl
# %matplotlib inline
import platform
import os
import ipympl
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import collections
import bisect
import seaborn as sns
import pickle
from scipy.stats import zscore
from tqdm import tqdm
from sklearn.decomposition import KernelPCA

In [51]:
def init_boc():
    '''
    Initialises and returns the BrainObservatoryCache
    
    Parameters
    ----------
    None
    
    Returns
    -------
    BrainObservatoryCache : allensdk.core.brain_observatory_nwb_data_set.BrainObservatoryNwbDataSet
    '''
    
    # Set file location based on platform. 
    platstring = platform.platform()
    if ('Darwin' in platstring) or ('macOS' in platstring):
        # macOS 
        data_root = "/Volumes/TReND2024/"
    elif 'Windows'  in platstring:
        # Windows (replace with the drive letter of USB drive)
        data_root = "E:/"
    elif ('amzn' in platstring):
        # then on Code Ocean
        data_root = "/data/"
    else:
        # then your own linux platform
        # EDIT location where you mounted hard drive
        data_root = "/media/$USERNAME/TReND2024/"

    from allensdk.core.brain_observatory_cache import BrainObservatoryCache

    manifest_file = os.path.join(data_root,'allen-brain-observatory/visual-coding-2p/manifest.json')

    return BrainObservatoryCache(manifest_file=manifest_file)
    

In [52]:
def get_session_ids(cre_line : list, imaging_depths : list, stimuli : list):
    '''
    Get lists of session_ids matching query parameters, grouped by cortical region
    
    Parameters
    ----------
    cre_line :  list
            Array of cre_lines to query
    imaging_depths : list
            Array of imaging_depths to query
    stimuli : list
            Array of stimuli to query
    Returns
    -------
    session_ids : Dictionary
            A Dictionary indexed by targeted_areas, coantaining arrays of matching sessions_ids
    '''
    session_ids = {}
    
    for area in boc.get_all_targeted_structures():
        exps = pd.DataFrame(boc.get_experiment_containers(
            targeted_structures=[area], cre_lines=cre_line, imaging_depths=imaging_depth
        ))
        experiment_container_ids = exps.id.to_numpy()
        sessions = boc.get_ophys_experiments(
            experiment_container_ids=experiment_container_ids, stimuli=stimuli
        )
        
        session_ids[area] = [sessions[i]['id'] for i in range(len(sessions))]
    
    return session_ids

In [53]:
def get_tables(data_set):
    '''
    Return the stimulus table and events table, clipped to the duration of the stimulus
    
    Parameters
    ----------
    data_set : allensdk.core.brain_observatory_nwb_data_set.BrainObservatoryNwbDataSet
    
    Returns
    -------
    stim_table : pandas DataFrame
            Dataframe containing stimulus information for each time step
    events : array_like
            2-D array of event values where events[n,m] is the event of cell n at time m
    '''
    stim_table = data_set.get_stimulus_table('natural_scenes')
    
    stim_start = stim_table.start.min()
    stim_end = stim_table.end.max()+1
    
    events = boc.get_ophys_experiment_events(ophys_experiment_id=session_id)
    events = events[:, stim_start:stim_end]
    
    stim_table.end = stim_table.end - stim_start
    stim_table.start = stim_table.start - stim_start
    
    return stim_table, events

In [54]:
def get_trial_respones(stim_table, events, axis : int = 1):
    '''
    Computes the average event value over the duration of each trial
    
    Parameters
    ----------
    stim_table : pandas DataFrame
            Dataframe containing stimulus information for each time step
    events : array_like
            2-D array of event values where events[n,m] is the event of cell n at time m
    axis : int, optional
            axis across which to average, default is 1
    Returns
    -------
    trial_responses : array_like
            2-D array of average event values where trial_responses[n,m]
            is the average event value of cell n for trial m
    '''
    averaging_indices = stim_table.end.to_numpy()
    cum_events = events.cumsum(axis=1)[:,averaging_indices]
    trial_sums = np.c_[cum_events[:,0] ,np.diff(cum_events, axis=1)]
    trial_responses = trial_sums/(stim_table.end.to_numpy() - stim_table.start.to_numpy() + 1)
    
    return trial_responses

In [55]:
def get_outlier_trials(trial_responses, z_score : float = 3.):
    '''
    Compute outlying trials based on symmetric z-score
    
    Parameters
    ----------
    trial_responses : pandas DataFrame
            Dataframe containing stimulus information for each time step
    z_score : float, optional
            z_score used to determine outliers with |p| > z_score, default is 3.0
    
    Returns
    -------
    outliers : array_like
            1-D array of trial indices corresponding to outliers
    '''
    trial_responses_df = pd.DataFrame(trial_responses.mean(axis=0))
    # Calculate the Z-score for each data point
    z_scores = trial_responses_df.apply(zscore)
    # Identify rows with any outlier (Z-score > 3 or Z-score < -3)
    outliers = (trial_responses_df[(z_scores > z_score) | (z_scores < -z_score)].to_numpy() > 0).nonzero()[0]
    
    return outliers

In [56]:
def get_stimulus_responses(trial_responses, stim_table, group_by : str, outliers = np.empty(0)):
    '''
    Computes the average trial_response for each value of the stimulus
    
    Parameters
    ----------
    trial_responses : pandas DataFrame
            Dataframe containing stimulus information for each time step
    outliers : array_like
            1-D array of trial indices corresponding to outliers, default is empty
    stim_table : pandas DataFrame
            Dataframe containing stimulus information for each time step
    sort_by : str
            Column name in stim_table used for grouping stimulus values for averaging
    Returns
    -------
    stimulus_responses : array_like
            2-D array of average trial_response values where stimulus_responses[n,m]
            is the average trial_response value of cell n stimulus m
    '''
    trial_responses = np.delete(trial_responses, outliers, axis=1)
    
    frames = stim_table[group_by].to_numpy()
    frames = np.delete(frames, outliers)
    
    sorting_indices = np.argsort(frames)
    sorted_events = trial_responses[:,sorting_indices]
    
    num_trials_per_stim = np.fromiter(collections.Counter(np.sort(frames)).values(), dtype=float)
    averaging_indices = num_trials_per_stim.cumsum().astype('int') - 1
    
    cum_trial_responses = sorted_events.cumsum(axis=1)[:,averaging_indices]
    stimulus_responses = np.c_[cum_trial_responses[:,0] ,np.diff(cum_trial_responses, axis=1)]/num_trials_per_stim
    
    return stimulus_responses

In [57]:
def PCA(stimulus_responses):
    '''
    Computes principle components using eigenvalue decomposition
    
    Parameters
    ----------
    stimulus_responses : array_like
            2-D array of average trial_response values where stimulus_responses[n,m]
            is the average trial_response value of cell n stimulus m
                        
    Returns
    -------
    D : array_like
            1_D array of eigenvalues, each repeated according to its multiplicity.
    U : array_like
            The unit eigenvectors, where U[:,n] is the eigenvector corresponding eigenvalue D[n]
    '''
    mu = np.mean(stimulus_responses,axis=0)
    stimulus_responses_centered = stimulus_responses - mu
    cov = np.cov(stimulus_responses_centered.T)
    
    # perform the eigendecomposition of the data covariance
    D, U = np.linalg.eig(cov) # D = vector of eigenvalues, U = eigenvectors matrix

    # make sure the eigenvalues are sorted (in descending order)
    indx = np.argsort(D)[::-1]
    D = D[indx]

    # arrange the eigenvectors according to the magnitude of the eigenvalues
    U = U[:,indx]
    
    return D, U

In [58]:
def get_dimensionality(D : np.ndarray):
    '''
    Computes number of dimensions required to explain required_variance
    
    Parameters
    ----------
    D : array_like
        1_D array of eigenvalues, each repeated according to its multiplicity.
    Returns
    -------
    dimension : float
        Dimension of the system computed according to 
    '''
    D = np.real(D)
#     dimension = (D.sum()**2) / (D @ D)
    
    explained_vars = D.cumsum() / D.sum()
    dimension = bisect.bisect_right(explained_vars, 0.95)
    return dimension

In [59]:
def plot_dimensions_by_region(dimensionality : dict, error_bar = 'se'):
    '''
    Plots dimensionality histogram by cortical region, with error bars,
    sorted by height in reverse order
    
    Parameters
    ----------
    dimensionality : dict
        A dictionary who's keys are visual regions (strings) and values are
        lists containing inate dimensions of neural spaces within that region
    error_bar : string, (string, number) tuple, callable or None
        Name of errorbar method (either “ci”, “pi”, “se”, or “sd”), or a tuple
        with a method name and a level parameter, or a function that maps from 
        a vector to a (min, max) interval, or None to hide errorbar
    kernel : str
        Name of kernel used for PCA, used for the title
    '''
    
    dimensionality = dict(sorted(dimensionality.items(), key=lambda x: np.mean(x[1]), reverse=True))
    
    colours = sns.color_palette("hls", 6)
    areas = ['VISal', 'VISam', 'VISl', 'VISp', 'VISpm', 'VISrl']
    
    palette = [colours[areas.index(area)] for area in dimensionality.keys()]
    
    sns.barplot(dimensionality, errorbar=(error_bar), capsize=.4, linewidth=2.5, palette=palette)
    plt.title(f"Dimensionality : 0.95 Variance")
    plt.savefig(f"dimensionality_pv.png")
    plt.close()

In [60]:
boc = init_boc()

In [61]:
# boc.get_all_cre_lines()

In [62]:
cre_line = boc.get_all_cre_lines()[0:2] # 'Cux2-CreERT2', 'Emx1-IRES-Cre''
# cre_line = [boc.get_all_cre_lines()[x] for x in [5,10,12]]
imaging_depth = boc.get_all_imaging_depths() # 175 - 250
stimulus = ['natural_scenes']

In [63]:
session_ids = get_session_ids(cre_line, imaging_depth, stimulus)

In [64]:
dimensionality = {}
for area in session_ids.keys():
    dimensions = []
    for session_id in tqdm(session_ids[area]):
        data_set = boc.get_ophys_experiment_data(ophys_experiment_id=session_id)
        stim_table, events = get_tables(data_set)
        trial_responses = get_trial_respones(stim_table, events)
        outliers = get_outlier_trials(trial_responses)
        stimulus_responses = get_stimulus_responses(trial_responses, stim_table, group_by='frame', outliers=outliers)
        D, U = PCA(stimulus_responses)
        dimension = get_dimensionality(D)
        dimensions.append(dimension)
    dimensionality[area] = dimensions
plot_dimensions_by_region(dimensionality)

100%|██████████| 21/21 [01:37<00:00,  4.62s/it]


In [65]:
# # Open a file and use dump() 
# with open('dimensionality_excitatory_2.pkl', 'wb') as file: 
#     # A new file will be created 
#     pickle.dump(dimensionality, file) 

In [66]:
sns.color_palette("hls", 6)

[(0.86, 0.3712, 0.33999999999999997),
 (0.8287999999999999, 0.86, 0.33999999999999997),
 (0.33999999999999997, 0.86, 0.3712),
 (0.33999999999999997, 0.8287999999999999, 0.86),
 (0.3712, 0.33999999999999997, 0.86),
 (0.86, 0.33999999999999997, 0.8287999999999999)]